In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pylab as plt
import matplotlib
%matplotlib qt
from numpy import linalg as la
from scipy.optimize import fsolve
from scipy import linalg as scpla
# import seaborn as sb
from mpl_toolkits.axes_grid1 import make_axes_locatable
from cmath import *
from mpl_toolkits.mplot3d import Axes3D
from scipy.optimize import fsolve,leastsq 
from math import tanh,cosh
import math
import time

from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

# from sympy import *
from scipy.linalg import schur, eigvals
from scipy.special import comb, perm

extras_require = {'PLOT':['matplotlib>=1.1.1,<3.0']},

## Help Functions

In [2]:
def generate_localstatsmat_eig_desym(Nparams,Grand,rho):
    '''
    Input
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    xaxiss,yaxiss =np.arange(NE*2),np.arange(NE*2)
    Xax,Yax=np.meshgrid(xaxiss,yaxiss)
    idxwhere = np.where(Xax>Yax)
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    X,Xsym,Xasym=np.zeros((N,N)),np.zeros((N,N)),np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    XT = X.copy().T

    Xsym = np.sqrt(rho)*X.copy()
    Xsym[idxwhere]=np.sqrt(rho)*XT[idxwhere]
    
    Xasym=np.sqrt(1-rho)*np.random.randn(NE*2,NE*2)
    X=Xsym.copy()+Xasym.copy()
    # components
    Xsym[:NE,:NE],Xsym[NE:,NE:]=xee*Xsym[:NE,:NE]/np.sqrt(NE),xii*Xsym[NE:,NE:]/np.sqrt(NI)
    Xsym[:NE,NE:],Xsym[NE:,:NE]=xei*Xsym[:NE,NE:]/np.sqrt(NE),xie*Xsym[NE:,:NE]/np.sqrt(NI)
    Xasym[:NE,:NE],Xasym[NE:,NE:]=xee*Xasym[:NE,:NE]/np.sqrt(NE),xii*Xasym[NE:,NE:]/np.sqrt(NI)
    Xasym[:NE,NE:],Xasym[NE:,:NE]=xei*Xasym[:NE,NE:]/np.sqrt(NE),xie*Xasym[NE:,:NE]/np.sqrt(NI)
    X=Xsym.copy()+Xasym.copy()
    # theoretical M (grandom) matrix
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    # for i in range(2):
    #     gmat[:,i]*=at[i]
    eigvgm,eigvecgm=la.eig(gmat)
    
    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii
    return (X,Xsym,Xasym,Gamp,eigvgm,eigvecgm,gaverage)

def generate_localstatsmat_eig(Nparams,Grand):
    '''
    Input 
      Grand 4, gee,gei,gie,gii
      Nparams: number of E and I neurons (same)
    '''
    xee,xei,xie,xii=Grand[0],Grand[1],Grand[2],Grand[3]
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    at = Nparams/N   
    X=np.zeros((N,N))
    X[:NE,:NE],X[NE:,NE:]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,NE:],X[NE:,:NE]=np.random.randn(NE,NE),np.random.randn(NE,NE)
    X[:NE,:NE],X[NE:,NE:]=xee*X[:NE,:NE]/np.sqrt(NE),xii*X[NE:,NE:]/np.sqrt(NI)
    X[:NE,NE:],X[NE:,:NE]=xei*X[:NE,NE:]/np.sqrt(NE),xie*X[NE:,:NE]/np.sqrt(NI)
    # theoretical M (grandom) matrix 
    gmat = np.array([[xee**2,xei**2],[xie**2,xii**2]])
    # first do not multiply at
    gaverage=0
    for i in range(2):
        for j in range(2):
            gaverage+=gmat[i,j]/2 # ntype=2
    gaverage=np.sqrt(gaverage)
    eigvgm,eigvecgm=la.eig(gmat)
    # properties
    Gamp=np.zeros((2,2))
    Gamp[0,0],Gamp[0,1],Gamp[1,0],Gamp[1,1]=xee,xei,xie,xii

    return (X,Gamp,eigvgm,eigvecgm,gaverage)

def generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII):
    # mean value 
    # first use rank-1 structure
    NE,NI=Nparams[0],Nparams[1]
    N=NE+NI
    at = Nparams/N   
    Am=np.zeros((N,N))
    Am[:NE,:NE],Am[:NE,NE:]=JEE/NE,-JEI/NI
    Am[NE:,:NE],Am[NE:,NE:]=JIE/NE,-JII/NI
    # ua,svam,vha=la.svd(Am)
    # va=vha.T
    # svam=svam[0]
    # ua,va=np.zeros((N,N)),np.zeros((N,N))
    # ua[:,0]=1.0/np.sqrt(N)
    # va[:NE,0]=JE/np.sqrt(NE)
    # va[NE:,0]=-JI/np.sqrt(NI)
    # va[:,0]/=-np.linalg.norm(va[:,0])
    # svam=np.sqrt(2*(JE**2+JI**2))
    # vha=va.T

    Jsv=np.zeros((2,2))
    Jsv[0,0],Jsv[0,1],Jsv[1,0],Jsv[1,1]=JEE,JEI,JIE,JII

    return (Am,Jsv)#,ua,svam,va)

# generate gmat, same gavg
def gmatamplitude_eig(gavgfix,typenum):
  Amplit = gavgfix*typenum
  numsample = typenum**2
  Amplitg= np.zeros(numsample)
  idxc=0
  while (1):
    if idxc>=numsample:
      Amplitg[numsample-1]=1.0-np.sum(Amplitg[:numsample-1])
      break
    p=np.random.random(1)
    Amplitg[idxc]=np.minimum(p,1.0-np.sum(Amplitg))
    if np.sum(Amplitg)>1.0:
      continue 
    elif np.sum(Amplitg)==1.0:
      break
    else:
      idxc +=1
      # Amplitg[idxc]=np.min(p,1.0-np.sum(Amplitg))
  # Amplitg=0
  Amplitg*=Amplit
  Amplitg=np.sqrt(Amplitg)
  return Amplitg

def confidence_ellipse(x, y, ax, n_std=3.0, facecolor='none', **kwargs):
    """
    Create a plot of the covariance confidence ellipse of `x` and `y`

    Parameters
    ----------
    x, y : array_like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    Returns
    -------
    matplotlib.patches.Ellipse

    Other parameters
    ----------------
    kwargs : `~matplotlib.patches.Patch` properties
    """
    if x.size != y.size:
        raise ValueError("x and y must be the same size")

    cov = np.cov(x, y)
    print(np.shape(cov))
    pearson = cov[0, 1]/np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse((0, 0),
        width=ell_radius_x * 2,
        height=ell_radius_y * 2,
        facecolor=facecolor,
        **kwargs)

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = np.mean(x)

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = np.mean(y)

    transf = transforms.Affine2D() \
        .rotate_deg(45) \
        .scale(scale_x, scale_y) \
        .translate(mean_x, mean_y)

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)


import math
import cmath
def ThreeSquare(x):
    if x.imag == 0:
        #real
        m = x.real
        ans = -math.pow(-m,1/3) if m<0 else math.pow(m,1/3)
    else:
        #complex
        ans = x**(1/3)
    return ans

# round to num
def RoundAns(x,num):
    if x.imag == 0:
        m = x.real
        ans = round(m,num)
    else:
        m = round(x.real,num)
        n = round(x.imag,num)
        ans = complex(m,n)
    return ans
    
def Cubic(args):
    a,b,c,d = args
    p = c/a-b**2/(3*a**2)
    q = d/a+2*b**3/(27*a**3)-b*c/(3*a**2)
    w = complex(-0.5,(3**0.5)/2)
    ww = complex(-0.5,-(3**0.5)/2)
    A = cmath.sqrt((q/2)**2+(p/3)**3)
    B = ThreeSquare(-q/2+A)
    C = ThreeSquare(-q/2-A)
    y1 = B+C
    y2 = w*B+ww*C
    y3 = ww*B+w*C
    D = b/(3*a)
    roots=[RoundAns(y1-D,6),RoundAns(y2-D,6),RoundAns(y3-D,6)]
    return roots

## For Case 3 and 4, with symmetric connectivity(FULL, HOMOGENEOUS SYMMETRY)
* Based on determinant lemma, calculate eigenvalue outlier (spectrum of eigenvalues)
* Based on perturbation theorem, calculate the statistics of eigenvectors (eigenvector perturbations, MEAN)
* Validate the results, in the case with homogeneous random gain and the case with heterogeneous random gain 

In [3]:
'''
WITH SYMMETRY
'''
'''
Cases 3,4 with symmetry
Focus on mean 
Focus on second moment
Focus on derivative on 
'''
# generate mean matrix
nrank=1
Nt=np.array([750,750])
NE,NI=Nt[0],Nt[1]
N=NE+NI
Nparams=np.array([NE,NI])
'''
>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
'''
nrank,ntrial,neta,ngavg=1,20,10,10

## for all casesq
xticks = np.linspace(-4.0,3.0,5)
xlims = [-4.1,3.1]
yticks = np.linspace(-1.3,1.3,3)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)

''' C0 rules '''
Xtest = np.zeros((NE*2,NE*2))
Xtest[:NE,:NE]=1
idxEE,idyEE=np.where(Xtest>0)
lowinEE=np.where(idxEE<idyEE)
upinEE=np.where(idxEE>idyEE)
idxUpEE,idyUpEE,idxDownEE,idyDownEE=idxEE[lowinEE],idyEE[lowinEE],idxEE[upinEE],idyEE[upinEE]
Xrep = np.zeros_like(Xtest)
Xrep[idxUpEE,idyUpEE]=1
idxUpII,idyUpII,idxDownII,idyDownII=idxUpEE+NE,idyUpEE+NE,idxDownEE+NE,idyDownEE+NE

''' 
---------- method 2 ------------------ 
'''
''' ## Three \bar{J} cases '''
JI,JE,a,b=3.5,1.0,0,0#1.0,3.5,0,0#3.5,1.00,-0.6,0.6 # already recording -3.5# 2.0
a,b=0.0,0.0
JEE,JIE,JEI,JII=JE+a,JE-a,JI-b,JI+b
''' Am -- J(g0=0), xAm(g0=0), yAm(g0=0) '''
Am,Jsv=generate_meanmat_eig(Nparams,JEE,JIE,JEI,JII)
# calculate the original \bar{m} and \bar{n}
eigvAm,eigveclAm,eigvecrAm=scpla.eig(Am,left=True,right=True)
meigvecAm=eigvecrAm.copy()
neigvecAm=eigveclAm.copy()
for i in range(nrank):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]*=(eigvAm[i]/normalizecoe)
for i in range(1,NE*2):
    normalizecoe=(np.sum(neigvecAm[:,i]*meigvecAm[:,i]))
    neigvecAm[:,i]/=normalizecoe
''' for reference '''
xAm,yAm=np.reshape(meigvecAm[:,0],(NE*2,1)),np.reshape(neigvecAm[:,0].copy()/eigvAm[0],(NE*2,1))

'''
##>>>>>>>>>>> g0!=0, one example >>>>>>>>
'''
gaverage=1.0
# X0=np.random.randn(NE*2,NE*2)*gaverage/np.sqrt(2*NE) # /np.sqrt(2) no later
X0=np.zeros((NE*2,NE*2))
X0[:NE,:NE]=np.reshape(np.random.randn(NE*NE),(NE,NE))*gaverage/np.sqrt(2*NE)
X0[NE:,NE:]=np.reshape(np.random.randn(NE*NE),(NE,NE))*gaverage/np.sqrt(2*NE)
X0[NE:,:NE]=np.reshape(np.random.randn(NE*NE),(NE,NE))*gaverage/np.sqrt(2*NE)
X0[:NE,NE:]=np.reshape(np.random.randn(NE*NE),(NE,NE))*gaverage/np.sqrt(2*NE)
Xsym=X0.copy()
XsymT=Xsym.copy().T
J0=Am.copy()+X0.copy()
## >>>>>> calculate the original \bar{m} and \bar{n} 
eigvJ0,eigrvecJ0=la.eig(J0)
inveigrvecJ0=la.inv(eigrvecJ0)
meig0 = np.squeeze(eigrvecJ0[:,:].copy())
neig0 = np.squeeze(inveigrvecJ0[:,:].copy()) # inverse
neig0=neig0.copy().T
## test neig0*meig0=1
print('test normal:',np.sum(neig0[:,0]*meig0[:,0]))
for i in range(nrank):
    neig0[:,i]*=eigvJ0[i]
for j in range(1,N):
    neig0[:,j]*=eigvJ0[j]
## >>>>> eigendecomposition 
leig0, reig0 = np.zeros((N,N)),np.zeros((N,N))
''' The first '''
reig0=meig0.copy()
normval=np.sum(reig0*neig0.copy(),axis=0)
normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
leig0=neig0.copy()/normval.copy()
''' calculate s0'''
Z = np.zeros((NE*2-1,NE*2-1))
for i in range(NE*2-1):
    b1=eigvJ0[i+1]
    Z[i,i]=1/(b1-eigvJ0[0])
S0=reig0[:,1:]@Z@leig0[:,1:].T
print('test normal 2:',np.sum(leig0[:,0]*reig0[:,0]))
print('test normal 3:',np.sum(meig0[:,0]*neig0[:,0]))
print('eigenvalues:',eigvJ0[:nrank])
'''## for reference '''
x0,y0=np.reshape(reig0[:,0],(NE*2,1)),np.reshape(leig0[:,0].copy(),(NE*2,1))
''' ## normalization 0 (reference) absolute reference is xAm, yAm '''
tildex,tildey=x0[:,0].copy(),y0[:,0].copy()
## make sure the E in y is positive, if negative then all x and y should change sign
## using y as reference, because the E and I parts in left eigenvector have opposite signs
if (np.mean(tildey[:NE])*yAm[0,0])<0:
    tildex*=(-1)
    tildey*=(-1)
x0,y0=np.reshape(tildex,(NE*2,1)),np.reshape(tildey,(NE*2,1))
## >>>> renormalization, mu_l and mu_r are unchanged
x0=np.sqrt(np.squeeze(tildey.T@xAm)/np.squeeze(yAm.T@tildex))*tildex
y0=np.sqrt(np.squeeze(tildex.T@yAm)/np.squeeze(xAm.T@tildey))*tildey
x0,y0=np.reshape(x0,(NE*2,1)),np.reshape(y0,(NE*2,1))
'''
##>>>>>>>>>>> end >>>>>>>>
'''

'''
Network Setting for Iterating Following
'''
signeta=np.ones(3)
# signeta[1]*=(-1)
## heterogeneous random gain
''' >>>>>>>>>>>>>> Case 1 and Case 2 >>>>>>>>>>>>>>>>>>>>>>> '''
xee,xei,xie,xii=1.0,1.0,1.0,1.0 ## HOMOgeneous random gain g_kl
# xee,xei,xie,xii=1.0,0.5,0.2,0.8 ## HETEROgeneous random gain
coeffeta=np.array([1.0,0.8,0.4]) ## HETERogeneous symmetry D_{\rho}
# coeffeta=np.array([1.0,0.8,1.0])
# coeffeta=np.array([1.0,1.0,1.0])

'''
Recording Variables
'''
# original data
Reigvecseries,Leigvecseries=np.zeros((ngavg,ntrial,NE*2),dtype=complex),np.zeros((ngavg,ntrial,NE*2),dtype=complex)
Beigvseries=np.zeros((ngavg,ntrial,N),dtype=complex)
''' All recorded connectivity statistics '''
statesReig,statesLeig=np.zeros((ngavg,ntrial,3,2),dtype=complex),np.zeros((ngavg,ntrial,3,2),dtype=complex) #E,I,Total/mean,std# 
coveigvecseries=np.zeros((ngavg,ntrial,3),dtype=complex) # E, I and total
''' All random matrices for each trials '''
## Not X total (symmetry), nut Y independent
gYbasexee=np.zeros((ntrial,NE*2,NE*2),dtype=complex)
x0series,y0series = np.zeros((ntrial,NE*2)),np.zeros((ntrial,NE*2))
''' Iterative Processing '''
etaseries=np.linspace(0.0,1.0,neta)
for iktrial in range(ntrial):
    '''
    ##>>>>>>>>>>> g0!=0, >>>>>>>>
    '''
    Xsym=np.random.randn(NE*2,NE*2)*gaverage/np.sqrt(2*NE) # /np.sqrt(2) no later
    Xsym[:NE,:NE]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    Xsym[:NE,NE:]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    Xsym[NE:,:NE]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    Xsym[NE:,NE:]= np.random.randn(NE,NE)*gaverage/np.sqrt(2*NE)
    XsymT=Xsym.copy().T
    X0 = Xsym.copy()  ## always make mistake here
    J0=Am.copy()+X0.copy()
    ''' # calculate the original \bar{m} and \bar{n} '''
    eigvJ0,eigrvecJ0=la.eig(J0)
    inveigrvecJ0=la.inv(eigrvecJ0)
    meig0 = np.squeeze(eigrvecJ0[:,:].copy())
    neig0 = np.squeeze(inveigrvecJ0[:,:].copy()) # inverse
    neig0=neig0.copy().T
    print('test normal:',np.sum(neig0[:,0]*meig0[:,0]))
    for i in range(nrank):
        neig0[:,i]*=eigvJ0[i]
    for j in range(1,N):
        neig0[:,j]*=eigvJ0[j]

    leig0, reig0 = np.zeros((N,N)),np.zeros((N,N))
    ''' The first '''
    reig0=meig0.copy()
    normval=np.sum(reig0*neig0.copy(),axis=0)
    normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
    leig0=neig0.copy()/normval.copy()
    ''' calculate s0'''
    Z = np.zeros((NE*2-1,NE*2-1))
    for i in range(NE*2-1):
        b1=eigvJ0[i+1]
        Z[i,i]=1/(b1-eigvJ0[0])
    S0=reig0[:,1:]@Z@leig0[:,1:].T
    print('test normal 2:',np.sum(leig0[:,0]*reig0[:,0]))
    print('test normal 3:',np.sum(meig0[:,0]*neig0[:,0]))
    print('eigenvalues:',eigvJ0[:nrank])
    '''## for reference '''
    x0,y0=np.reshape(reig0[:,0],(NE*2,1)),np.reshape(leig0[:,0].copy(),(NE*2,1))
    ''' ## normalization 0 (reference) absolute reference is xAm, yAm '''
    tildex,tildey=np.reshape(x0[:,0].copy(),(NE*2,1)),np.reshape(y0[:,0].copy(),(NE*2,1))
    ## make sure the E in y is positive, if negative then all x and y should change sign
    if (np.mean(tildey[:NE])*yAm[0,0])<0:   # keep the same
        tildex*=(-1)
        tildey*=(-1)
    x0,y0=np.reshape(tildex,(NE*2,1)),np.reshape(tildey,(NE*2,1))
    x0=np.sqrt(np.squeeze(tildey.T@xAm)/np.squeeze(yAm.T@tildex))*tildex
    y0=np.sqrt(np.squeeze(tildex.T@yAm)/np.squeeze(xAm.T@tildey))*tildey
    x0,y0=np.reshape(x0,(NE*2,1)),np.reshape(y0,(NE*2,1))
    ## record 
    x0series[iktrial,:],y0series[iktrial,:]=x0[:,0],y0[:,0]
    for idxeta,eta in enumerate(etaseries):
        etaset=eta*np.ones(3)#np.zeros(3)#eta*np.ones(3)
        for icoeff in range(3):
          etaset[icoeff]*=coeffeta[icoeff]
        # etaset[0]*=0.1
        Xinit = Xsym.copy()

        ''' heterogeneous ETA method 1 '''
        ## EE ##
        asqr=(1-np.sqrt(1-etaset[0]**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
        aamp=np.sqrt(asqr)
        Xinit[:NE,:NE]=signeta[0]*aamp*XsymT[:NE,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[:NE,:NE].copy()
        ## EI IE##
        asqr=(1-np.sqrt(1-etaset[1]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:,:NE]=signeta[1]*aamp*XsymT[NE:,:NE].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,:NE].copy()
        Xinit[:NE,NE:]=signeta[1]*aamp*XsymT[:NE,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[:NE,NE:].copy()
        ## II ##
        asqr=(1-np.sqrt(1-etaset[2]**2))/2.0
        aamp=np.sqrt(asqr)
        Xinit[NE:,NE:]=signeta[2]*aamp*XsymT[NE:,NE:].copy()+np.sqrt(1-aamp**2)*Xsym[NE:,NE:].copy()
        # ### >>>>>>>>. half Exc sym >>>>>>>>>.
        # hNE=int(5*NE/3)
        # asqr=(1-np.sqrt(1-eta**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
        # aamp=np.sqrt(asqr)
        # Xinit[:hNE,:hNE]=signeta[0]*aamp*XsymT[:hNE,:hNE].copy()+np.sqrt(1-aamp**2)*Xsym[:hNE,:hNE].copy()
        ''' THEORETICAL FULL ETA '''
        symtheofull=gaverage**2*(etaset[0]*signeta[0]+etaset[2]*signeta[2]+2*etaset[1]*signeta[1])/4.0#+(a-b)**2/(NE*2)
        X=Xinit.copy()
        # incase heterogeneous
        X[:NE,:NE]*=(xee)
        X[:NE,NE:]*=(xei)
        X[NE:,:NE]*=(xie)
        X[NE:,NE:]*=(xii)

        # overall
        J = X.copy()+Am.copy()
        JT = J.copy().T
        symnumfull=np.sum(np.sum(J*JT))/(4*NE**2)-np.mean(np.mean(J.copy()))**2
        symnumfull*=2*NE
        print('compare degree of symmetry:',symnumfull,'--num; ',etaset*signeta,'--setting')
        # properties
        eigvJ,eigrvecJ=la.eig(J)
        inveigrvecJ=la.inv(eigrvecJ)
        meig = np.squeeze(eigrvecJ[:,:].copy())
        # meig = np.reshape(meig,(NE*2,nrank))
        neig = np.squeeze(inveigrvecJ[:,:].copy()) # inverse
        neig=neig.copy().T
        print('test normal 4:',np.sum(neig[:,0]*meig[:,0]))
        for i in range(nrank):
            neig[:,i]*=eigvJ[i]
        for j in range(1,N):
            neig[:,j]*=eigvJ[j]
        '''
        Sort Eigenvalue in ascending 
        '''
        eigenvalue_Rsort = np.squeeze(-np.abs(np.real(eigvJ.copy())))  ## for lambda_0<0, argsort from large to small -np.abs
        idxsort=np.argsort(eigenvalue_Rsort)
        eigenvalue_sort = np.squeeze(eigvJ.copy())
        # eigvalseries[idxeta,iktrial,:] = eigenvalue_sort[idxsort]
        eigvJ= eigvJ[idxsort]  #### >>>>>>>>>>>>>> resorting >>>>>>>>>
        meig = meig[:,idxsort] #### >>>>>>>>>>>>>> resorting >>>>>>>>>
        neig = neig[:,idxsort] #### >>>>>>>>>>>>>> resorting >>>>>>>>>
        ## >>> for eigendecomposition 
        Leig, Reig = np.zeros((N,N)),np.zeros((N,N))
        Reig = meig.copy()
        Leig = neig.copy()
        normval=np.sum(Reig*Leig.copy(),axis=0)
        normval=np.repeat(np.reshape(normval,(1,N)),N,axis=0)
        Leig=Leig.copy()/normval.copy()  # note that here we change from neig --> Leig, otherwise, if we use copy, there'll be some sort of mistakes

        '''
        normalization 0 (reference)
        '''
        tildex,tildey=Reig[:,0].copy(),Leig[:,0].copy()
        if (np.mean(tildey[:NE])*yAm[0,0])<0:
            tildex*=(-1)
            tildey*=(-1)
        xnorm0=np.sqrt(np.squeeze(tildey.T@x0)/np.squeeze(y0.T@tildex))*tildex
        ynorm0=np.sqrt(np.squeeze(tildex.T@y0)/np.squeeze(x0.T@tildey))*tildey

        Beigvseries[idxeta,iktrial,:]=eigvJ.copy()
        Reigvecseries[idxeta,iktrial,:],Leigvecseries[idxeta,iktrial,:]=xnorm0[:].copy(),ynorm0[:].copy()

        '''  RECORDING STATES '''
        statesReig[idxeta,iktrial,0,0],statesReig[idxeta,iktrial,1,0]=np.mean(Reigvecseries[idxeta,iktrial,:NE]),np.mean(Reigvecseries[idxeta,iktrial,NE:])
        statesReig[idxeta,iktrial,2,0]=np.mean(Reigvecseries[idxeta,iktrial,:])
        statesReig[idxeta,iktrial,0,1],statesReig[idxeta,iktrial,1,1]=np.std(Reigvecseries[idxeta,iktrial,:NE]),np.std(Reigvecseries[idxeta,iktrial,NE:])
        statesReig[idxeta,iktrial,2,1]=np.std(Reigvecseries[idxeta,iktrial,:])
       
        statesLeig[idxeta,iktrial,0,0],statesLeig[idxeta,iktrial,1,0]=np.mean(Leigvecseries[idxeta,iktrial,:NE]),np.mean(Leigvecseries[idxeta,iktrial,NE:])
        statesLeig[idxeta,iktrial,2,0]=np.mean(Leigvecseries[idxeta,iktrial,:])
        statesLeig[idxeta,iktrial,0,1],statesLeig[idxeta,iktrial,1,1]=np.std(Leigvecseries[idxeta,iktrial,:NE]),np.std(Leigvecseries[idxeta,iktrial,NE:])
        statesLeig[idxeta,iktrial,2,1]=np.std(Leigvecseries[idxeta,iktrial,:])

        ''' covariance '''
        '''
        Note, we should change the normalization for the first item to the (Refer) normalization 
        tildeReig,tildeLeig
        normalization 0
        '''
        ## normalization 0 reference
        tildeReig,tildeLeig=np.squeeze(Reigvecseries[idxeta,iktrial,:]),np.squeeze(Leigvecseries[idxeta,iktrial,:])
        reig,leig=np.reshape(tildeReig[:].copy(),(NE*2,1)),np.reshape(tildeLeig[:].copy(),(NE*2,1))
        ## mean at eact point
        rEceneig,rIceneig = np.mean(reig[:NE]),np.mean(reig[NE:])
        lEceneig,lIceneig = np.mean(leig[:NE]),np.mean(leig[NE:])
        ## random components
        rrandeig,lrandeig=reig.copy(),leig.copy()
        rrandeig[:NE],rrandeig[NE:]=rrandeig[:NE]-rEceneig,rrandeig[NE:]-rIceneig
        lrandeig[:NE],lrandeig[NE:]=lrandeig[:NE]-lEceneig,lrandeig[NE:]-lIceneig
        coveigvecseries[idxeta,iktrial,0]=np.reshape(rrandeig[:NE],(NE,1)).T@np.reshape(lrandeig[:NE],(NE,1))
        coveigvecseries[idxeta,iktrial,1]=np.reshape(rrandeig[NE:],(NE,1)).T@np.reshape(lrandeig[NE:],(NE,1))
        coveigvecseries[idxeta,iktrial,2]=coveigvecseries[idxeta,iktrial,0]+coveigvecseries[idxeta,iktrial,1]
        ''' record the random derivative '''
        if idxeta == 0:
            print('recording!!!')
            ## log: there is a bug, X is the total random xconn, including symmetry
            gYbasexee[iktrial,:,:]=Xsym.copy()


33333 0.26666667] --setting
test normal 4: (1.0000000000000315+1.6215774154579086e-14j)
compare degree of symmetry: 0.5831489487837755 --num;  [0.77777778 0.62222222 0.31111111] --setting
test normal 4: (0.9999999999999851+7.133783316260792e-14j)
compare degree of symmetry: 0.6663513899654496 --num;  [0.88888889 0.71111111 0.35555556] --setting
test normal 4: (1.0000000000000098+8.79893350355521e-16j)
compare degree of symmetry: 0.7495529018397211 --num;  [1.  0.8 0.4] --setting
test normal 4: (0.9999999999999496-2.898537653724396e-14j)
test normal: (0.9999999999999203+6.414066835274863e-15j)
test normal 2: (1.0000000000000007-6.310887241768095e-30j)
test normal 3: (-2.5421944264905436-1.630580495977496e-14j)
eigenvalues: [-2.54219443+0.j]
compare degree of symmetry: 0.0003526542280020911 --num;  [0. 0. 0.] --setting
test normal 4: (0.9999999999999203+6.414066835274863e-15j)
recording!!!
compare degree of symmetry: 0.08371569005560242 --num;  [0.11111111 0.08888889 0.04444444] --settin

* Recording PARAMETERS, VARIABLES and DATA

In [ ]:
# saving data
now = time.strftime("%Y-%m-%d-%H_%M_%S",time.localtime(time.time()))
Jparameters = np.zeros(4+1)
Jparameters[0],Jparameters[1],Jparameters[2],Jparameters[3]=JEE,JEI,JIE,JII
Jparameters[4]=NE
Randgain=np.zeros(4)
Randgain[0],Randgain[1],Randgain[2],Randgain[3]=xee,xei,xie,xii

np.savez(now+'_perturbationHetereta_data',x0series=x0series,y0series=y0series,xAm=xAm,yAm=yAm,Jparameters=Jparameters,Randgain=Randgain,gaverage=gaverage,etaseries=etaseries,etaset=etaset,signeta=signeta,Reigvecseries=Reigvecseries[:,:,:,0],Leigvecseries=Leigvecseries[:,:,:,0],Beigvseries=Beigvseries[:,:,:3],meigvecAm=meigvecAm[:,0],neigvecAm=neigvecAm[:,0],eigvAm=eigvAm,statesReig=statesReig,statesLeig=statesLeig,coveigvecseries=coveigvecseries,gYbasexee=gYbasexee,coeffeta=coeffeta)


* eigenvalue spectrum (ensemble averaging)

In [5]:
'''
with symmetry, 
eigvJ0
'''
idxgavg,idxtrial=5,8 # 6,3,0
figtspt,axtspt=plt.subplots(figsize=(6,3))
cm='bgc'
idxc=0
axtspt.scatter(np.real(Beigvseries[idxgavg,idxtrial,1:]),np.imag(Beigvseries[idxgavg,idxtrial,1:]),s=5,c=cm[idxc],alpha=0.25) # >>>>>>>>>>>>>>
axtspt.scatter(np.real(Beigvseries[idxgavg,:,0]),np.imag(Beigvseries[idxgavg,:,0])-0.2*idxc,s=20,c=cm[idxc],marker='^',alpha=0.5) # >>>>>>>>>>>>
axtspt.set_aspect('equal')
axtspt.plot(np.ones(2)*eigvAm[0],np.array([-1.0,1.0]),linestyle='--',linewidth=1.5,color='grey')
axtspt.spines['right'].set_color('none')
axtspt.spines['top'].set_color('none')
# X axis location
axtspt.xaxis.set_ticks_position('bottom')
axtspt.spines['bottom'].set_position(('data', 0))

aee,aei,aie,aii=xee,xei,xie,xii
print(xee,xei,xie,xii)
idxtrial=10
eta=0
xticks = np.linspace(-1.3,1.3,2)
xlims = [-1.3,1.3]
yticks = np.linspace(-1.3,1.3,2)
ylims = [-1.31,1.31]
theta = np.linspace(0, 2 * np.pi, 200)
# first do not multiply at
ahomo=gaverage
xee_,xei_,xie_,xii_=ahomo*aee/np.sqrt(2),ahomo*aei/np.sqrt(2),ahomo*aie/np.sqrt(2),ahomo*aii/np.sqrt(2)
gmat = np.array([[xee_**2,xei_**2],[xie_**2,xii_**2]])
gaverage_=0
for i in range(2):
    for j in range(2):
        gaverage_+=gmat[i,j]/2 # ntype=2
gaverage_=np.sqrt(gaverage_)
eigvgm,eigvecgm=la.eig(gmat) 
r_g2=np.max(eigvgm)
r_g = np.sqrt(r_g2)
print(r_g)
## >>> Now we have symmetry >>
eta=etaseries[idxgavg]
longaxis,shortaxis=(1+eta)*r_g ,(1-eta)*r_g 
xr = longaxis*np.cos(theta)
yr = shortaxis*np.sin(theta)
axtspt.plot(xr, yr, color="gray", linewidth=2.5,linestyle='--',label=r'ellipse') # >>>>>
axtspt.plot(longaxis*np.ones(2), np.array([-1.0,1.0]), color="gray", linewidth=2.5,linestyle='--') # >>>>>
axtspt.set_title(r'$\eta=$'+np.str(eta))
xticks = np.linspace(-3.5,2.5,2)
xlims = [-3.5,2.5]
yticks = np.linspace(-1.0,1.0,2)
ylims = [-1.1,1.1]
axtspt.set_xlim(xlims)
axtspt.set_ylim(ylims)
axtspt.set_xticks(xticks)
axtspt.set_yticks(yticks)
axtspt.set_aspect('equal')

axtspt.plot(xr, yr, color="pink", linewidth=2.5,linestyle='--',label=r'ellipse') # >>>>>
axtspt.plot(longaxis*np.ones(2), np.array([-1.0,1.0]), color="pink", linewidth=2.5,linestyle='--') # >>>>>
axtspt.set_title(r'$\eta=$'+np.str(eta))
xticks = np.linspace(-3.5,2.5,2)
xlims = [-3.5,2.5]
yticks = np.linspace(-1.0,1.0,2)
ylims = [-1.1,1.1]
axtspt.set_xlim(xlims)
axtspt.set_ylim(ylims)
axtspt.set_xticks(xticks)
axtspt.set_yticks(yticks)
axtspt.set_aspect('equal')

1.0 1.0 1.0 1.0
0.9999999999999999
D:\UseApp\Anaconda3\lib\site-packages\numpy\core\_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


* Population statistics(for backup)
    * eigenvalue outlier 
    * overlap between left and right eigenvectors

In [11]:
'''
determinant lemma, odd terms 
even order terms should be maitained
'''
xticks = np.linspace(-2.91,-2.5,2)
xlims = [-2.91,-2.5]
yticks = np.linspace(-2.91,-2.5,2)
ylims = [-2.91,-2.5]

coeff=coeffeta.copy()
nAm=np.reshape(yAm*eigvAm[0],(NE*2,1)) # lambda*left eigenvector -- n not l
mAm=np.reshape(xAm.copy(),(NE*2,1))
nAm,mAm=np.real(nAm),np.real(mAm)
lambda_theo2 = np.zeros(neta)
lambda_theo4 = np.zeros(neta)
lambda_num = np.mean(np.squeeze(Beigvseries[:,:,0]),axis=1)
cutoff = 2

for idxeta,eta in enumerate(etaseries):
    etaE,etaB,etaI=eta*coeff[0],eta*coeff[1],eta*coeff[2]
    '''
    Even-order terms have basic element XX.T(X.TX)
    '''
    bXXT=np.eye(NE*2)
    bXXT[:NE,:NE]*=(etaE*signeta[0]+etaB*signeta[1])/4
    bXXT[NE:,NE:]*=(etaI*signeta[2]+etaB*signeta[1])/4
    # cut off = 2
    C=comb(2,1)
    print('second moment item-num.:',C)
    B2=C*(nAm.T@la.matrix_power(bXXT,1)@mAm)*gaverage**2. ## gaverage to the 2
    coeff2 = [1,-eigvAm[0],0,-B2]  ## lambda_0 -- eigvAm
    C=comb(4,2)
    print('fourth moment item-num.:',C)
    B4=C*(nAm.T@la.matrix_power(bXXT,2)@mAm)*gaverage**4 ## gaverage to the 4
    coeff4 = [1,-eigvAm[0],0,-B2,0,-B4]
    ''' soluter np.roots '''
    # lambda_test2=np.roots(coeff2)
    # lambda_theo2[idxeta]=lambda_test2[0]
    ''' Cubic Equation '''
    roots = Cubic(coeff2)
    lambda_theo2[idxeta]=np.real(roots[0])
    lambda_test4=np.roots(coeff4)
    lambda_theo4[idxeta]=lambda_test4[0]

# ## >>>>>> eigenvalue outlier theo V.S. num    
# fig,ax=plt.subplots(figsize=(5,5))  
# ax.plot(lambda_num,lambda_theo2,'b',label='2 cut off')
# ax.plot(lambda_num,lambda_theo4,'r',label='4 cut off')
# ax.plot(lambda_num,lambda_num,'k',label='real')
# # ax.set_xlim(xlims)
# # ax.set_ylim(ylims)
# # ax.set_xticks(xticks)
# # ax.set_yticks(yticks)
# # ax.set_aspect('equal')
# ax.legend()

## >>>>> eigenvalue outlier V.S. degree of symmetry
fig,ax=plt.subplots(figsize=(5,3))
ax.plot(etaseries,lambda_theo2[:],'c',label='2 cut off')
ax.plot(etaseries,lambda_theo4[:],'g',label='2 cut off')
ax.plot(etaseries,lambda_num,'b',label='2 cut off')
ax.set_aspect('equal')

second moment item-num.: 2.0
fourth moment item-num.: 6.0
second moment item-num.: 2.0
fourth moment item-num.: 6.0
second moment item-num.: 2.0
fourth moment item-num.: 6.0
second moment item-num.: 2.0
fourth moment item-num.: 6.0
D:\UseApp\Anaconda3\lib\site-packages\numpy\lib\polynomial.py:238: ComplexWarning: Casting complex values to real discards the imaginary part
  p = p.astype(float)
<ipython-input-11-a063d2aec5ec>:43: ComplexWarning: Casting complex values to real discards the imaginary part
  lambda_theo4[idxeta]=lambda_test4[0]
second moment item-num.: 2.0
fourth moment item-num.: 6.0
second moment item-num.: 2.0
fourth moment item-num.: 6.0
second moment item-num.: 2.0
fourth moment item-num.: 6.0
second moment item-num.: 2.0
fourth moment item-num.: 6.0
second moment item-num.: 2.0
fourth moment item-num.: 6.0
second moment item-num.: 2.0
fourth moment item-num.: 6.0
D:\UseApp\Anaconda3\lib\site-packages\numpy\core\_asarray.py:85: ComplexWarning: Casting complex values to

In [12]:

'''
determinant lemma, odd terms 
even order terms should be maitained
combination

!!! In sparse condition
'''
import numpy.polynomial.polynomial as poly
from scipy.special import comb, perm
coeffB4 = np.zeros(neta)
envelopeRe = np.zeros(neta)

yticks = np.linspace(-2.50,1.5,2)
ylims = [-2.6,1.6]
xticks = np.linspace(-0.,1.,2)
xlims = [-0.1,1.1]

nAm=np.reshape(yAm.copy()*eigvAm[0],(NE*2,1))
mAm=np.reshape(xAm.copy(),(NE*2,1))
nAm,mAm=np.real(nAm),np.real(mAm)
lambda_theo2 = np.zeros((neta,3),dtype=complex)
nAmsparse,mAmsparse = nAm.copy(),mAm.copy()
lambda_num = np.transpose(np.squeeze(Beigvseries[:,:,:2]),(2,1,0))
for idxeta,eta in enumerate(etaseries[:]):
    etaset = eta*np.ones(3)
    for i in range(3):
        etaset[i]*=coeffeta[i]

    '''
    Even-order terms have basic element XX.T(X.TX)
    '''
    bXXTE,bXXTI=np.eye(NE),np.eye(NE)
    bXXTE*=(etaset[0]*signeta[0]+etaset[1]*signeta[1])/4
    bXXTI*=(etaset[1]*signeta[1]+ etaset[2]*signeta[2])/4
    # cut off = 2
    C=comb(2,1)
    print(C)
    moment2=np.zeros((NE*2,NE*2))
    moment2[:NE,:NE]=(bXXTE)
    moment2[NE:,NE:]=bXXTI
    B2=np.squeeze(C*(nAm.T@(moment2)@mAm)*gaverage**2)
    roots = Cubic([1,-eigvAm[0],0,-B2])
    coeffB4[idxeta]=-B2
    envelopeRe[idxeta]=np.mean(-np.abs(np.squeeze(Beigvseries[idxeta,:,2])))
    lambda_theo2[idxeta,:]=roots


# fig,ax=plt.subplots(figsize=(5,3))  
for i in range(3):
    ax.plot(etaseries,lambda_theo2[:,i],color='r',linewidth=1.5,label='theo')
# fig,ax=plt.subplots(figsize=(5,3))  
# ax.plot(coeffB4,color=cm[i],linewidth=1.5,label='theo')
# ax.set_xlim(xlims)
# ax.set_ylim(ylims)
# ax.set_xticks(xticks)
# ax.set_yticks(yticks)
# ax.set_aspect('equal')
# ax.legend()

2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
2.0
D:\UseApp\Anaconda3\lib\site-packages\numpy\core\_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
D:\UseApp\Anaconda3\lib\site-packages\numpy\core\_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
D:\UseApp\Anaconda3\lib\site-packages\numpy\core\_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


* overall perturbation -- $\Delta_g + \Delta_{\rho}$

In [13]:
'''
Total Theoretical Solution for Covariance
'''

'''
in the case with reciprical connectivity,
with certain degree of homogeneous symmetry

we integrating from 0 to eta and get the CHANGING 
'''
DeltaLR_matheta,DeltaLR_mathg,DeltaLR_num = np.zeros((neta,3),dtype=complex),np.zeros((neta,3),dtype=complex),np.zeros((neta,ntrial,3),dtype=complex)
DeltaLR_eigv = np.zeros((neta,ntrial),dtype=complex)

Jinit = np.zeros((ntrial,NE*2,NE*2),dtype=complex)
## Jinit same for identical trial # realization
Leigvecinit,Reigvecinit = np.zeros((ntrial,NE*2),dtype=complex),np.zeros((ntrial,NE*2),dtype=complex)

for itrial in range(ntrial):
    # initial eigenvectors
    l0,r0=np.reshape(Leigvecseries[0,itrial,:].copy(),(2*NE,1)),np.reshape(Reigvecseries[0,itrial,:].copy(),(2*NE,1))#np.reshape(-yAm,(2*NE,1)),np.reshape(-xAm,(2*NE,1))#

    for idxeta in range(neta):
        ## choose one specific degree of symmetry
        eta = etaseries[idxeta]
        etaE,etaEI,etaI=eta*coeffeta[0],eta*coeffeta[1],eta*coeffeta[2]
        rhoE,rhoEI,rhoI= (1-np.sqrt(1-etaE**2))/2.0,(1-np.sqrt(1-etaEI**2))/2.0,(1-np.sqrt(1-etaI))/2.0
        print('etaset:',etaE,etaEI,etaI)

        print('rhoset:',rhoE,rhoEI,rhoI)

        # numerical value
        changeL_num = np.reshape(np.squeeze(Leigvecseries[idxeta,itrial,:].copy()),(NE*2,1))-np.reshape(yAm,(2*NE,1))#-l0#np.mean(np.squeeze(Leigvecseries[idxeta,itrial,:]))#np.reshape(np.squeeze(Leigvecseries[0,itrial,:]),(NE*2,1))
        # changeL_num[:numE1,0]-=np.mean(changeL_num[:numE1,0])
        # changeL_num[numE1:NE,0]-=np.mean(changeL_num[numE1:NE,0])
        # changeL_num[NE:,0]-=np.mean(changeL_num[NE:,0])
        changeR_num = np.reshape(np.squeeze(Reigvecseries[idxeta,itrial,:].copy()),(NE*2,1))-np.reshape(xAm,(2*NE,1))#-r0#np.mean(np.squeeze(Reigvecseries[idxeta,itrial,:]))#np.reshape(np.squeeze(Reigvecseries[0,itrial,:]),(NE*2,1))
        # changeR_num[:numE1,0]-=np.mean(changeR_num[:numE1,0])
        # changeR_num[numE1:NE,0]-=np.mean(changeR_num[numE1:NE,0])
        # changeR_num[NE:,0]-=np.mean(changeR_num[NE:,0])

        DeltaLR_num[idxeta,itrial,2]=np.squeeze(changeL_num.T@changeR_num)
        DeltaLR_num[idxeta,itrial,0]=np.sum(changeL_num[:NE,0]*changeR_num[:NE,0])
        DeltaLR_num[idxeta,itrial,1]=np.sum(changeL_num[NE:,0]*changeR_num[NE:,0])
   
        # EXCITATORY
        # JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercent[0]**2)*(np.sqrt((1-rhoE)*rhoE)-np.sqrt(rhoE)    
        # -JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(ppercent[1]**2)*(np.sqrt((1-rhoB)*rhoB)-np.sqrt(rhoB)        bXXTE1,bXXTE2,bXXTII=np.eye(numE1),np.eye(numE2),np.eye(NE)

        '''
        ### >>>>>>>>>>> delta in eta
        '''       
        DeltaLR_matheta[idxeta,0]=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(etaE-2.0*np.sqrt(rhoE))*signeta[0]/4.0
        DeltaLR_matheta[idxeta,0]+=(-JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(etaEI-2.0*np.sqrt(rhoEI))*signeta[1]/4.0)
        print('DeltaE1:',DeltaLR_matheta[idxeta,0])

        DeltaLR_matheta[idxeta,1]=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(etaEI-2.0*np.sqrt(rhoEI))*signeta[1]/4.0 
        DeltaLR_matheta[idxeta,1]+=(-JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(etaI-2.0*np.sqrt(rhoI))*signeta[2]/4.0)
        '''
        ### >>>>>>>>>>> delta in g with eta
        '''  
        DeltaLR_mathg[idxeta,0]=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(np.sqrt(rhoE))*signeta[0]/2.0
        DeltaLR_mathg[idxeta,0]+=(-JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(np.sqrt(rhoEI))*signeta[1]/2.0)
        print('DeltaE1_g:',DeltaLR_mathg[idxeta,0])


        DeltaLR_mathg[idxeta,1]=JE*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(np.sqrt(rhoEI))*signeta[1]/2.0 
        DeltaLR_mathg[idxeta,1]+=(-JI*gaverage**2/(JE-JI)/(eigvAm[0]**2)*(np.sqrt(rhoI))*signeta[2]/2.0)

        DeltaLR_matheta[idxeta,2]=2*DeltaLR_matheta[idxeta,1]+2*DeltaLR_matheta[idxeta,0]
        DeltaLR_mathg[idxeta,2]=2*DeltaLR_mathg[idxeta,1]+2*DeltaLR_mathg[idxeta,0]

        DeltaLR_eigv[idxeta,itrial]= (Beigvseries[idxeta,itrial,0]-eigvAm[0])/eigvAm[0]

: 0.052096791761191696 0.03267242567082018 0.04661764970881854
DeltaE1: (-0.00014066631723881536+0j)
DeltaE1_g: (0.012940666317238716+0j)
etaset: 0.5555555555555556 0.4444444444444445 0.22222222222222224
rhoset: 0.08426029035845095 0.052096791761191696 0.059041448155901566
DeltaE1: (-0.00027484883253737016+0j)
DeltaE1_g: (0.016274848832537245+0j)
etaset: 0.6666666666666666 0.5333333333333333 0.26666666666666666
rhoset: 0.12732200375003505 0.07704741531834935 0.07182558071116235
DeltaE1: (-0.0004699992527703435+0j)
DeltaE1_g: (0.019669999252770194+0j)
etaset: 0.7777777777777777 0.6222222222222222 0.3111111111111111
rhoset: 0.18573031947264546 0.10857966769314359 0.0850033467337089
DeltaE1: (-0.000714739543722837+0j)
DeltaE1_g: (0.02311473954372266+0j)
etaset: 0.8888888888888888 0.7111111111111111 0.35555555555555557
rhoset: 0.2709385763545744 0.14846017734319167 0.09861351404025681
DeltaE1: (-0.000897630719025926+0j)
DeltaE1_g: (0.026497630719025726+0j)
etaset: 1.0 0.8 0.4
rhoset: 0.5 0

In [14]:
fig,ax=plt.subplots(figsize=(5,5))
cl='rbk'
ax.plot(etaseries,np.mean(np.squeeze(DeltaLR_num[:,:,2]),axis=1),color=cl[0],linewidth=1.5)
ax.plot(etaseries,np.mean(np.squeeze(DeltaLR_eigv[:,:]),axis=1),color=cl[1],linewidth=1.5)
ax.plot(etaseries,DeltaLR_matheta[:,2]+DeltaLR_mathg[:,2],color=cl[2],linewidth=1.5)

D:\UseApp\Anaconda3\lib\site-packages\numpy\core\_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
D:\UseApp\Anaconda3\lib\site-packages\numpy\core\_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
D:\UseApp\Anaconda3\lib\site-packages\numpy\core\_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)


## Back up code

* First attemp, overlap/covariance between -- direction of change

In [ ]:
'''
variance of r and variance of l <r.Tr>, <l.T,l>
overlap between r and l <l.Tr>
'''
xticks = np.linspace(-0.01,0.1,2)
xlims = [-0.01,0.1]
yticks = np.linspace(-0.01,0.1,2)
ylims = [-0.01,0.1]


itrial=14
gY=np.squeeze(gYbasexee[itrial,:,:])
## >>>>> in the case with symmetry, the first eta point corresponds to eta = 0, which is just the original random matrix without symmetry
x0,y0=np.reshape(Reigvecseries[0,itrial,:,0],(NE*2,1)),np.reshape(Leigvecseries[0,itrial,:,0],(NE*2,1))

'''
rho_prime=eta/(2*sqrt(1-eta**2))
rho'/2/sqrt(rho) to 2nd=(1+np.sqrt(1-etaseries**2))/8/(1-etaseries**2)
rho'/2/sqrt(1-rho) to 2nd=(1-np.sqrt(1-etaseries**2))/8/(1-etaseries**2)
'''
heta=etaseries[1]-etaseries[0]
rhoseries=(1-np.sqrt(1-etaseries**2))/2.0 ## when eta = 0, asqr = 0, aamp = 0, XT-0, X-1
sqrt_1meta2= np.sqrt(1-etaseries**2)
coeffsqrtY,coeffsqrtYT = (1-sqrt_1meta2)/sqrt_1meta2**2/8,(1+sqrt_1meta2)/sqrt_1meta2**2/8
overlapperturb,overlapvector = np.zeros((neta,2,2,NE),dtype=complex),np.zeros((neta,2,2,NE),dtype=complex)
# EI,LR,:
for i in range(1,neta-1):
    Jprimemat=np.sqrt(coeffsqrtYT[i])*gY.copy().T-np.sqrt(coeffsqrtY[i])*gY
    leftperturb=np.squeeze(-np.squeeze(y0.T@Jprimemat))/(-eigvAm[0])# left perturbation E
    rightperturb=np.squeeze(-np.squeeze(Jprimemat@x0))/(-eigvAm[0])#*Beigvseries[i,itrial,0]# right perturbation E
    overlapperturb[i,0,0,:],overlapperturb[i,0,1,:]=leftperturb[:NE],rightperturb[:NE]
    overlapperturb[i,1,0,:],overlapperturb[i,1,1,:]=leftperturb[NE:],rightperturb[NE:]
    overlapvector[i,0,0,:]=np.squeeze(Leigvecseries[i,itrial,:NE,0])-np.mean(np.squeeze(Leigvecseries[i,itrial,:NE,0]))
    overlapvector[i,1,0,:]=np.squeeze(Leigvecseries[i,itrial,NE:,0])-np.mean(np.squeeze(Leigvecseries[i,itrial,NE:,0]))
    overlapvector[i,0,1,:]=np.squeeze(Reigvecseries[i,itrial,:NE,0])-np.mean(np.squeeze(Reigvecseries[i,itrial,:NE,0]))
    overlapvector[i,1,1,:]=np.squeeze(Reigvecseries[i,itrial,NE:,0])-np.mean(np.squeeze(Reigvecseries[i,itrial,NE:,0]))
## E population 
# vectorcovE,perturbationcovE = np.zeros((neta,2,2)),np.zeros((neta,2,2))
directionvecE,directionvecI,directionperturbE,directionperturbI=np.zeros((neta,2,2)),np.zeros((neta,2,2)),np.zeros((neta,2,2)),np.zeros((neta,2,2))
ampvecE,ampvecI,ampperturbE,ampperturbI=np.zeros((neta,2)),np.zeros((neta,2)),np.zeros((neta,2)),np.zeros((neta,2))
for i in range(neta):
    vectorEmat,vectorImat=np.squeeze(overlapvector[i,0,:,:]),np.squeeze(overlapvector[i,1,:,:])
    perturbEmat,perturbImat=np.squeeze(overlapperturb[i,0,:,:]),np.squeeze(overlapperturb[i,1,:,:])
    vcovE,vcovI=vectorEmat@vectorEmat.T,vectorImat@vectorImat.T
    pcovE,pcovI=perturbEmat@perturbEmat.T ,perturbImat@perturbImat.T

    vvE,dirvE=la.eig(vcovE)
    vvI,dirvI=la.eig(vcovI)
    vpE,dirpE=la.eig(pcovE)
    vpI,dirpI=la.eig(pcovI)
    directionvecE[i,:,:],directionvecI[i,:,:]=dirvE,dirvI
    directionperturbE[i,:,:],directionperturbI[i,:,:]=dirpE,dirpI
    ampvecE[i,:],ampvecI[i,:]=vvE,vvI
    ampperturbE[i,:],ampperturbI[i,:]=vpE,vpI

# confidence_ellipse(Leftperturb,Rightperturb,ax,edgecolor='orange')
# confidence_ellipse(np.real(nIvec),np.real(mIvec),ax,edgecolor='green')

In [ ]:

fig,ax=plt.subplots(figsize=(5,5)) 
for idxeta in np.arange(1,8):
    heta=0.1
    ax.plot([0,0+ampvecE[idxeta,0]*directionvecE[idxeta,0,0]],[0,0+ampvecE[idxeta,0]*directionvecE[idxeta,1,0]],color='r',linestyle='-',linewidth=1.5)
    ax.plot([0,0+ampvecI[idxeta,0]*directionvecI[idxeta,0,0]],[0,0+ampvecI[idxeta,0]*directionvecI[idxeta,1,0]],color='orange',linestyle='-',linewidth=1.5)
    # ax.plot([0,0+ampvecE[idxeta+1,0]*directionvecE[idxeta+1,0,0]],[0,0+ampvecE[idxeta+1,0]*directionvecE[idxeta+1,1,0]],color='r',linestyle='-',linewidth=1.5)
    # ax.plot([0,0+ampvecI[idxeta+1,0]*directionvecI[idxeta+1,0,0]],[0,0+ampvecI[idxeta+1,0]*directionvecI[idxeta+1,1,0]],color='orange',linestyle='-',linewidth=1.5)

    vecEx,vecEy=ampvecE[idxeta,0]*directionvecE[idxeta,0,0],ampvecE[idxeta,0]*directionvecE[idxeta,1,0]
    vecIx,vecIy=ampvecI[idxeta,0]*directionvecI[idxeta,0,0],ampvecI[idxeta,0]*directionvecI[idxeta,1,0]
    ax.plot([vecEx,vecEx+ampperturbE[idxeta,1]*directionperturbE[idxeta,0,1]*heta],[vecEy,vecEy+ampperturbE[idxeta,1]*directionperturbE[idxeta,1,1]*heta],color='b',linestyle='--',linewidth=1.5)
    ax.plot([vecIx,vecIx+ampperturbI[idxeta,1]*directionperturbI[idxeta,0,1]*heta],[vecIy,vecIy+ampperturbI[idxeta,1]*directionperturbI[idxeta,1,1]*heta],color='g',linestyle='--',linewidth=1.5)

xticks = np.linspace(-0.0,0.15,2)
xlims = [-0.00,0.15]
yticks = np.linspace(-0.0,0.035,2)
ylims = [-0.0,0.025]
ax.set_xlim(xlims)
ax.set_ylim(ylims)
ax.set_xticks(xticks)
ax.set_yticks(yticks)
# ax.set_aspect('equal')



# fig,ax=plt.subplots(figsize=(5,5))  
# ax.scatter(overlapperturb[idxeta,0,0,:],overlapperturb[idxeta,0,1,:],s=5,c='orange',alpha=0.5)
# ax.scatter(Leigvecseries[idxeta,itrial,:NE,0]-np.mean(Leigvecseries[idxeta,itrial,:NE,0]),Reigvecseries[idxeta,itrial,:NE,0]-np.mean(Reigvecseries[idxeta,itrial,:NE,0]),s=5,c='blue',alpha=0.5)
# xticks = np.linspace(-0.06,0.06,2)
# xlims = [-0.06,0.06]
# yticks = np.linspace(-0.06,0.06,2)
# ylims = [-0.06,0.06]
# ax.set_xlim(xlims)
# ax.set_ylim(ylims)
# ax.set_xticks(xticks)
# ax.set_yticks(yticks)
# ax.set_aspect('equal')

# fig,ax=plt.subplots(figsize=(5,5))  
# ax.scatter(overlapperturb[idxeta,1,0,:],overlapperturb[idxeta,1,1,:],s=5,c='orange',alpha=0.5)
# ax.scatter(Leigvecseries[idxeta,itrial,NE:,0]-np.mean(Leigvecseries[idxeta,itrial,NE:,0]),Reigvecseries[idxeta,itrial,NE:,0]-np.mean(Reigvecseries[idxeta,itrial,NE:,0]),s=5,c='blue',alpha=0.5)
# ax.set_aspect('equal')
# ax.set_xlim(xlims)
# ax.set_ylim(ylims)
# ax.set_xticks(xticks)
# ax.set_yticks(yticks)
# ax.set_aspect('equal')

h=etaseries[1]-etaseries[0]
fig,ax=plt.subplots(figsize=(5,5))  
ax.scatter(overlapperturb[idxeta,0,0,:]*h,Leigvecseries[idxeta,itrial,:NE,0]-(Leigvecseries[idxeta-1,itrial,:NE,0]),s=5,c='orange',alpha=0.5)
ax.scatter(overlapperturb[idxeta,1,0,:]*h,Leigvecseries[idxeta,itrial,NE:,0]-(Leigvecseries[idxeta-1,itrial,NE:,0]),s=5,c='blue',alpha=0.5)
ax.set_aspect('equal')

fig,ax=plt.subplots(figsize=(5,5))  
ax.scatter(overlapperturb[idxeta,0,1,:]*h,Reigvecseries[idxeta,itrial,:NE,0]-(Reigvecseries[idxeta-1,itrial,:NE,0]),s=5,c='orange',alpha=0.5)
ax.scatter(overlapperturb[idxeta,1,1,:]*h,Reigvecseries[idxeta,itrial,NE:,0]-(Reigvecseries[idxeta-1,itrial,NE:,0]),s=5,c='blue',alpha=0.5)
ax.set_aspect('equal')